## **Fase 1 - Extract**
*En esta fase inicial, el enfoque está en la extracción de datos brutos directamente desde la plataforma de Airbnb. Utilizando un scraper automatizado, extraemos información esencial sobre las propiedades disponibles, como ubicación, precios, tipo de alojamiento, disponibilidad y reseñas de los usuarios. La extracción es un paso crucial para asegurar que todos los datos relevantes se reúnan y estén listos para su análisis. Nos aseguramos de que los datos se extraigan de forma precisa y consistente, lo cual es fundamental para el éxito de las fases siguientes del proceso. El objetivo de esta fase es consolidar la información extraída en un formato accesible (DataFrame) de manera eficiente y automatizada, para que posteriormente puedan ser limpiados y transformados.* **(necesita edicion - version test)**

In [2]:
# ETL imports
import requests 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager



# General 
from datetime import datetime
import re
import time
from time import sleep
import pandas as pd
import numpy as np
import json          
import os
import tqdm
from tqdm import tqdm 

In [3]:
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36")

In [4]:
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=opts)

In [5]:
# Lista de distritos de Barcelona
distritos = [
    "Ciutat Vella",
    "Eixample",
    "Sants-Montjuïc",
    "Les Corts",
    "Sarrià-Sant Gervasi",
    "Gràcia",
    "Horta-Guinardó",
    "Nou Barris",
    "Sant Andreu",
    "Sant Martí"
]

# Creamos una lista para almacenar enlaces de distritos
link_distrito = []

# URL base con el formato de Airbnb
url_base = "https://www.airbnb.es/s/{distrito}--Barcelona--España/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&monthly_start_date=2024-11-01&monthly_length=3&monthly_end_date=2025-02-01&price_filter_input_type=0&channel=EXPLORE&query={distrito}%2C%20Barcelona%2C%20España&date_picker_type=calendar&source=structured_search_input_header&search_type=autocomplete_click"

# Iterar sobre cada distrito y generar el enlace
for distrito in distritos:
    # Reemplazar espacios por %20 y caracteres especiales
    distrito_url = distrito.replace(" ", "%20").replace("à", "%C3%A0").replace("ç", "%C3%A7").replace("í", "%C3%AD")
    
    # Formatear el enlace con el nombre del distrito
    enlace = url_base.format(distrito=distrito_url)

    link_distrito.append(enlace)

# Lista vacía para almacenar los links de habitaciones
links_to_scrapp = []

# Accedemos a las paginas de cada distrito
for distrito in tqdm(link_distrito, desc="Accediendo a distritos"):
    
    driver.get(distrito)

    # Pausa
    sleep(3)

    # Bucle para la paginación y extración
    while len(links_to_scrapp) < 500:  # Limitar a 500 enlaces
        try:
            sleep(2)
            # Obtener el código fuente de la página actual
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')

            # Extraer todos los enlaces de habitaciones en la página actual
            link_habitacion = driver.find_elements(By.CSS_SELECTOR, 'a[href*="/rooms/"]')

            # Lista de urls target
            links_target = []

            # Recorrer todos los enlaces y filtrar los que contienen '/rooms'
            for _ in link_habitacion:
                link = _.get_attribute('href')
                if link and '/rooms/' in link:
                    links_target.append(link)

            # Eliminar duplicados en la lista de enlaces
            links_filtered = set(links_target)
            print(f"Número de links filtrados en la página: {len(links_filtered)}")

            # Añadimos los links en la lista de scrappeo
            for url in links_filtered:
                if len(links_to_scrapp) < 500:  # Limitar a 500 enlaces
                    links_to_scrapp.append(url)
                    print(f'Agregando URL: {url}')
                else:
                    break

            print(f"Total de links extraídos hasta ahora: {len(links_to_scrapp)}")

            if len(links_to_scrapp) >= 500:  
                print('Total de enlaces alcanzado. Deteniendo la extracción......')
                break

            # Pausa antes de hacer click en la próxima página
            sleep(2)

            # Buscar y hacer click en el botón "Siguiente"
            next_button = driver.find_element(By.CSS_SELECTOR, 'a[aria-label="Siguiente"]')
            next_button.click()
            print('Siguiente página...')

        except Exception as e:
            print('No se pudo cargar la siguiente página o no hay más páginas.')
            print(e)
            break

# Imprimir a lista final de enlaces
print("Extracción completa. Total de links:", len(links_to_scrapp))

Accediendo a distritos:   0%|          | 0/10 [00:00<?, ?it/s]

Número de links filtrados en la página: 18
Agregando URL: https://www.airbnb.es/rooms/1270266469340006214?adults=1&category_tag=Tag%3A8678&children=0&enable_m3_private_room=true&infants=0&pets=0&photo_id=2015148890&search_mode=regular_search&check_in=2024-11-25&check_out=2024-11-30&source_impression_id=p3_1730726872_P3vj_Ongl2sdAaWK&previous_page_section_name=1000&federated_search_id=474377fe-633e-4c6f-a11b-e5609cfd007a
Agregando URL: https://www.airbnb.es/rooms/1243254147120959358?adults=1&category_tag=Tag%3A8678&children=0&enable_m3_private_room=true&infants=0&pets=0&photo_id=2015735133&search_mode=regular_search&check_in=2024-11-10&check_out=2024-11-15&source_impression_id=p3_1730726872_P38QcbvHB2UsEAkh&previous_page_section_name=1000&federated_search_id=474377fe-633e-4c6f-a11b-e5609cfd007a
Agregando URL: https://www.airbnb.es/rooms/1268163818980729724?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-12-01&check_out=2024-12-06

Accediendo a distritos:  10%|█         | 1/10 [01:15<11:20, 75.59s/it]

No se pudo cargar la siguiente página o no hay más páginas.
Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[aria-label="Siguiente"]"}
  (Session info: chrome=130.0.6723.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x005638B3+24035]
	(No symbol) [0x004EBC44]
	(No symbol) [0x003CC2D3]
	(No symbol) [0x0040DC86]
	(No symbol) [0x0040DECB]
	(No symbol) [0x0044B9D2]
	(No symbol) [0x0042FED4]
	(No symbol) [0x0044953F]
	(No symbol) [0x0042FC26]
	(No symbol) [0x0040218C]
	(No symbol) [0x0040310D]
	GetHandleVerifier [0x008096D3+2800643]
	GetHandleVerifier [0x0086428E+3172286]
	GetHandleVerifier [0x0085CEA2+3142610]
	GetHandleVerifier [0x00606C60+692624]
	(No symbol) [0x004F4C5D]
	(No symbol) [0x004F1968]
	(No symbol) [0x004F1B00]
	(No symbol) [0x004E3FB0]
	BaseThreadInitThunk [0x74FE7BA9+25]
	RtlInitializeExceptionChain

Accediendo a distritos:  20%|██        | 2/10 [02:18<09:02, 67.84s/it]

Número de links filtrados en la página: 18
Agregando URL: https://www.airbnb.es/rooms/1242680451877593023?adults=1&category_tag=Tag%3A8678&children=0&enable_m3_private_room=true&infants=0&pets=0&photo_id=1999572795&search_mode=regular_search&check_in=2024-11-04&check_out=2024-11-09&source_impression_id=p3_1730727005_P3375HKyCS2j8RT5&previous_page_section_name=1000&federated_search_id=5bc98de9-5c60-4bc1-8304-a28789aaf412
Agregando URL: https://www.airbnb.es/rooms/41822352?adults=1&category_tag=Tag%3A8678&children=0&enable_m3_private_room=true&infants=0&pets=0&photo_id=1877977358&search_mode=regular_search&check_in=2024-11-27&check_out=2024-12-02&source_impression_id=p3_1730727005_P3_f0S_wUDsOrLcW&previous_page_section_name=1000&federated_search_id=5bc98de9-5c60-4bc1-8304-a28789aaf412
Agregando URL: https://www.airbnb.es/rooms/24156044?adults=1&category_tag=Tag%3A8678&children=0&enable_m3_private_room=true&infants=0&pets=0&photo_id=1629445617&search_mode=regular_search&check_in=2024-12-0

Accediendo a distritos: 100%|██████████| 10/10 [02:58<00:00, 17.80s/it]

Extracción completa. Total de links: 500


In [6]:
now = datetime.now()

# Función para extraer el texto de un elemento dado
def extract_text(soup, tag, class_name, index=0, default=np.nan):
    try:
        elements = soup.find_all(tag, class_=class_name)
        return elements[index].text.strip() if elements else default  # Retorna el texto del elemento especificado
    except (AttributeError, IndexError):
        return default

# Diccionario para los campos a extraer, organizados en el orden deseado
fields = {
    "Titles": ("span", "l1h825yc", 1),  # Título del alojamiento
    "Host_name": ("div", "t1pxe1a4", 0),  # Nombre del anfitrión
    "Property_types": ("h2", "hpipapi", 0),  # Tipo de propiedad
    "Prices_per_night": ("span", "_11jcbg2", 0),  # Precio por noche
    "Check_ins": ("div", "i1303y2k", 0),  # Check-in
    "Check_outs": ("div", "i1303y2k", 1),  # Check-out
    "Location": ("div", "_152qbzi", 0),  # Ubicación
}

# Limitar a 500 enlaces para la muestra
sample_links = links_to_scrapp[:500]

# Lista para almacenar los datos extraídos
data_list = []

# Iterar sobre cada enlace en la muestra
for link in tqdm(sample_links, desc="Extrayendo datos"):
    if 'adults' not in link:
        # Verificar si ya hay parámetros en la URL
        if '?' in link:
            link += '&adults=1'  # Agregar & si ya hay otros parámetros
        else:
            link += '?adults=1'  # Agregar ? si no hay parámetros

    print(f"Accediendo a: {link}")        

    driver.get(link)  # Acceder a la página
    sleep(5)  # Espera para cargar la página

    # Obtener el objeto BeautifulSoup
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')


    try:
        # Boton de traduccion

        button_trad = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Cerrar']"))
        )
        button_trad.click()
        print("Botón de traducción clicado con éxito.")

    except (NoSuchElementException, TimeoutException):
        print("No se encontró el botón de traducción.")
    

        # Capturar el contenido de la página -----------> Provisional
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    data = {
    'urls': link,
    'timestamp': now.strftime('%Y-%m-%d %H:%M:%S'),  # Añadir la fecha y hora formateada
    'record_id': link.split('/')[-1].split('?')[0]  # Usar el ID de la URL como record_id
    }

    # Extraer usando el diccionario y la función de extracción
    data.update({field: extract_text(soup, *details) for field, details in fields.items()})

    # Obtener las reseñas y calificaciones usando XPATH
    ratings_xpaths = [
        "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/div/section/div[3]/div[2]",
        "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[1]/div/div[2]/div/div/div/a/div/div[6]/div[1]",
        "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/div/section/div[3]/div[2]",
    ]

    reviews_xpaths = [
        "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/div/section/div[3]/a",
        "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[1]/div/div[2]/div/div/div/a/div/div[10]/div[1]",
        "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/div/section/div[3]/a",
    ]

    rating = num_reviews = np.nan  # Inicializa como NaN

    # Intentar obtener las calificaciones
    for xpath in ratings_xpaths:
        try:
            rating = driver.find_element(By.XPATH, xpath).text
            break  # Salir del bucle si se encuentra la calificación
        except:
            continue  # Continuar intentando con el siguiente XPATH

    # Intentar obtener el número de reseñas
    for xpath in reviews_xpaths:
        try:
            num_reviews_text = driver.find_element(By.XPATH, xpath).text
            if 'evaluaciones' in num_reviews_text:
                num_reviews = num_reviews_text.split()[0]  # Captura el número de evaluaciones
                break  # Salir del bucle si se encuentra el número de evaluaciones
        except:
            continue  # Continuar intentando con el siguiente XPATH

    # Añadir calificaciones y número de reseñas al diccionario de datos
    data['Ratings'] = rating
    data['Num_reviews'] = num_reviews
    
    # Extraccion de cleaning fee
    try:
        # Buscar el elemento de gastos de limpieza
        cleaning_fee_loc = driver.find_element(By.CSS_SELECTOR, "div[data-plugin-in-point-id='BOOK_IT_SIDEBAR'] div._m495dq")
        
        # Buscar el dato de la tarifa con expresión regular
        gastos_limpieza = re.search(r"Gastos de limpieza.*?(\d+ €)", cleaning_fee_loc.text, re.DOTALL)
        
        if gastos_limpieza:
            cleaning_fee = gastos_limpieza.group(1)
        else:
            print("No se encontraron los gastos de limpieza.")
            cleaning_fee = "No hay gasto"  # Indicar que no se encontraron gastos de limpieza

    except Exception as e:
        print(f"No se pudo encontrar el elemento de gastos de limpieza: {e}")
        cleaning_fee = "Elemento no encontrado"  # Indicar que el elemento no estaba en la página

    # Añadir el cleaning_fee al diccionario de datos
    data['Cleaning_fee'] = cleaning_fee
    

    # Extracción de detalles (camas, baños, etc.) usando el selector CSS
    details = driver.find_elements(By.CSS_SELECTOR, "div.o1kjrihn ol.lgx66tx")
    detalles = [_.text for _ in details]

    # Procesar cada detalle y categorizar
    detalles_limpio = {}
    for i in detalles:
        for fragment in i.split(" · "):
            # Detecta y asigna cada categoría por palabras clave en cada fragmento
            if "viajero" in fragment:
                detalles_limpio['Viajeros'] = fragment
            elif "dormitorio" in fragment:
                detalles_limpio['Dormitorios'] = fragment
            elif "cama" in fragment:
                detalles_limpio['Camas'] = fragment
            elif "baño" in fragment or "Baño" in fragment:
                detalles_limpio['Baños'] = fragment
            else:
                detalles_limpio['Otros'] = fragment

    # Agregar detalles categorizados al diccionario de datos sin afectar los campos existentes
    data.update(detalles_limpio)

    # Agregar el número máximo de huéspedes
    n_guests = driver.find_elements(By.CSS_SELECTOR, "div[data-plugin-in-point-id='POLICIES_DEFAULT'] div.i1303y2k span")
    guests_count = np.nan  # Valor predeterminado en caso de no encontrar
    for guest in n_guests:
        if 'huésped' in guest.text or 'huéspedes' in guest.text:
            guests_count = guest.text.split()[1]  # Extraer solo el número de huéspedes
            break  # Salir del bucle si se encuentra el número de huéspedes

    # Añadir el número máximo de huéspedes al diccionario de datos
    data['Maximum_guests'] = guests_count

    # Añadir los datos extraídos a la lista
    data_list.append(data)

# Crear el DataFrame final con los datos extraídos
df = pd.DataFrame(data_list)

Extrayendo datos:   0%|          | 0/500 [00:00<?, ?it/s]

Accediendo a: https://www.airbnb.es/rooms/1270266469340006214?adults=1&category_tag=Tag%3A8678&children=0&enable_m3_private_room=true&infants=0&pets=0&photo_id=2015148890&search_mode=regular_search&check_in=2024-11-25&check_out=2024-11-30&source_impression_id=p3_1730726872_P3vj_Ongl2sdAaWK&previous_page_section_name=1000&federated_search_id=474377fe-633e-4c6f-a11b-e5609cfd007a


Extrayendo datos:   0%|          | 1/500 [00:08<1:09:54,  8.41s/it]

No se encontró el botón de traducción.
Accediendo a: https://www.airbnb.es/rooms/1243254147120959358?adults=1&category_tag=Tag%3A8678&children=0&enable_m3_private_room=true&infants=0&pets=0&photo_id=2015735133&search_mode=regular_search&check_in=2024-11-10&check_out=2024-11-15&source_impression_id=p3_1730726872_P38QcbvHB2UsEAkh&previous_page_section_name=1000&federated_search_id=474377fe-633e-4c6f-a11b-e5609cfd007a


Extrayendo datos:   0%|          | 2/500 [00:13<55:20,  6.67s/it]  

...

No se encontró el botón de traducción.
Accediendo a: https://www.airbnb.es/rooms/10205793?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-11-30&check_out=2024-12-05&source_impression_id=p3_1730727005_P3_xfmC4L9HBSl7h&previous_page_section_name=1000&federated_search_id=5bc98de9-5c60-4bc1-8304-a28789aaf412


Extrayendo datos: 100%|█████████▉| 499/500 [1:10:20<00:08,  8.48s/it]

No se encontró el botón de traducción.
Accediendo a: https://www.airbnb.es/rooms/2555964?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2025-01-03&check_out=2025-01-08&source_impression_id=p3_1730727005_P3damnx3URP_Uejc&previous_page_section_name=1000&federated_search_id=5bc98de9-5c60-4bc1-8304-a28789aaf412


Extrayendo datos: 100%|██████████| 500/500 [1:10:29<00:00,  8.46s/it]

No se encontró el botón de traducción.


In [7]:
df

,urls,timestamp,record_id,Titles,Host_name,Property_types,Prices_per_night,Check_ins,Check_outs,Cleaning_fees,Location,Ratings,Num_reviews,Camas,Baños,Maximum_guests,Viajeros,Dormitorios,Otros
0,https://www.airbnb.es/rooms/127026646934000621...,2024-11-04 14:30:46,1270266469340006214,Cancelación gratuita antes del 24 nov. Si canc...,Anfitrión: Joar,"Habitación en Barcelona, España",46 €,Llegada a partir de las 10:00,Salida antes de las 12:00,10 €,"Barcelona, Catalunya, España",Nuevo,NaN,1 cama,Baño compartido,2,NaN,NaN,NaN
1,https://www.airbnb.es/rooms/124325414712095935...,2024-11-04 14:30:46,1243254147120959358,Este alojamiento tiene una ubicación estratégi...,Anfitrión: Valentina Sofia,"Habitación en Barcelona, España",45 €,Horario de llegada: de 15:00 a 23:00,Salida antes de las 11:00,20 €,"Barcelona, Catalunya, España",Nuevo,NaN,1 cama,Baño compartido,NaN,NaN,NaN,NaN
2,https://www.airbnb.es/rooms/126816381898072972...,2024-11-04 14:30:46,1268163818980729724,¡Bienvenido a tu escapada perfecta en el coraz...,Anfitrión: Joonas,"Alojamiento entero: apartamento en Barcelona, ...",147 €,Llegada a partir de las 15:00,Salida antes de las 12:00,126 €,"Barcelona, Catalunya, España",Nuevo,NaN,2 camas,1 baño,4,4 viajeros,2 dormitorios,NaN
3,https://www.airbnb.es/rooms/44247318?adults=1&...,2024-11-04 14:30:46,44247318,ACOGEDOR APARTAMENTO DE PLAYA DE 1 DORMITORIO ...,Anfitrión: Jan,"Alojamiento entero: apartamento en Barcelona, ...",55 €,Horario de llegada: de 18:00 a 23:00,Salida antes de las 11:00,80 €,"Barcelona, Catalunya, España","4,68",19,1 cama,1 baño,2,2 viajeros,1 dormitorio,NaN
4,https://www.airbnb.es/rooms/119269213083576009...,2024-11-04 14:30:46,1192692130835760091,Disfruta de la privacidad de este alojamiento...,Anfitrión: Ele,"Habitación privada en: loft en Barcelona, España",56 €,Horario de llegada: de 14:00 a 21:00,Salida antes de las 12:00,48 €,"Barcelona, Catalunya, España","5,0",4,2 camas,1 baño privado,1,1 viajero,1 dormitorio,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,https://www.airbnb.es/rooms/21722132?adults=1&...,2024-11-04 14:30:46,21722132,El apartamento Batlló Balocnies se encuentra e...,Anfitrión: Habitat Apartments,"Alojamiento entero: apartamento en Barcelona, ...",115 €,Horario de llegada: de 15:00 a 21:00,Salida antes de las 11:00,NaN,NaN,"4,31",62,2 camas,1 baño,3,3 viajeros,1 dormitorio,NaN
496,https://www.airbnb.es/rooms/572182415885999147...,2024-11-04 14:30:46,572182415885999147,"Muy bien ubicado: centro ciudad, con muchas at...",Anfitrión: Pau,"Alojamiento entero: apartamento en Barcelona, ...",149 €,Horario de llegada: de 16:00 a 21:00,Salida antes de las 11:00,70 €,NaN,"4,81",43,2 camas,1 baño,2,2 viajeros,1 dormitorio,NaN
497,https://www.airbnb.es/rooms/590503?adults=1&ch...,2024-11-04 14:30:46,590503,Apartamento tipo LOFT en ZONA MUY SEGURA y muy...,Anfitrión: Alex,"Alojamiento entero: apartamento en Barcelona, ...",86 €,Horario de llegada: de 15:00 a 0:00,Salida antes de las 11:00,60 €,"Barcelona, España",NaN,NaN,1 cama,1 baño,5,5 viajeros,2 dormitorios,NaN
498,https://www.airbnb.es/rooms/10205793?adults=1&...,2024-11-04 14:30:46,10205793,Precioso apartamento situado en la mejor zona ...,Anfitrión: Pilar,"Alojamiento entero: apartamento en Barcelona, ...",112 €,Llegada flexible,Salida antes de las 11:00,50 €,NaN,"4,73",187,3 camas,1 baño,6,6 viajeros,3 dormitorios,NaN


In [10]:
df.shape

(500, 19)

In [14]:
df.to_csv('df_sample_V3.csv', index = False)

In [8]:
df = pd.read_csv('df_sample_V2')
df

FileNotFoundError: [Errno 2] No such file or directory: 'df_sample_V2'